<a href="https://colab.research.google.com/github/SofiiaBobr/Data-science/blob/main/%D0%91%D0%BE%D0%B1%D1%80%D1%96%D0%B2%D0%B5%D1%86%D1%8C_%D0%A1%D0%BE%D1%84%D1%96%D1%8F_%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D1%94_%D0%B7%D0%B0%D0%B2%D0%B4%D0%B0%D0%BD%D0%BD%D1%8F_8(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 Домашнє завдання №8
**Заняття 23. Методи машинного навчання — Частина 3**

❗ У назві файлу має бути ваше ім’я з навчальної платформи.

---
Для цієї роботи ми будемо використовувати 📎 дані: [`my_phone.csv`](https://github.com/robot-dreams-code/DATA-SCIENCE-PROFESSION-04/blob/main/%D0%97%D0%B0%D0%BD%D1%8F%D1%82%D1%82%D1%8F%2023/my_phone.csv)

---

## 🔍 Сценарій   
Ви плануєте створити власну лінійку мобільних телефонів. Щоб краще орієнтуватись у специфікаціях та цінах, ви зібрали дані з сайту Rozetka. Частина рядків у датасеті — телефони, які ви хочете створити самостійно, і для них відсутня інформація про ціну (price_range). Ваше завдання — дослідити ці дані, знайти аномалії, а потім навчити моделі передбачати ціну і класифікувати ваші нові телефони.

## 📁 Опис даних

Дані містять характеристики телефонів:

| Колонка         | Опис                                                 |
|-----------------|------------------------------------------------------|
| `battery_power` | Потужність батареї (mAh)                             |
| `bluetooth`     | Підтримка Bluetooth (1 — так, 0 — ні)                |
| `weight`        | Вага телефону                                        |
| `memory`        | Обсяг пам’яті (внутрішньої)                          |
| `n_cores`       | Кількість ядер процесора                            |
| `ram`           | Обсяг оперативної пам’яті                            |
| `dual_sim`      | Підтримка 2 SIM-карт (1 — так, 0 — ні)               |
| `wifi`          | Підтримка Wi-Fi (1 — так, 0 — ні)                    |
| `pixel_height`  | Висота екрану в пікселях                            |
| `pixel_width`   | Ширина екрану в пікселях                             |
| `price_range`   | Цінова категорія: (0 — дуже низька, 1 — низька, 2 — середня,3 — висока) ⬇⬇⬇ |

---
&nbsp;

 **❗️❗️❗️ `price_range` може бути порожньою — це ваші майбутні моделі телефонів, для яких ви хотече спрогнозувати ціну. НЕ ВИДАЛЯЙТЕ ЇХ❗️**

## Завдання 1: Дослідження та очищення даних — 15 балів

📌 Зробіть повний аналіз:
- `.info()`, `.describe()`, `.isnull().sum()`
- Побудуйте графіки, перевірте логіку значень

🔍 Знайдіть 5 кардинально різних **підступів у даних**.   
*P.S.* Присутність NA в колонці 1 та присутність NA в колонці 2 НЕ Є кардинально різними підступами.

&nbsp;

### ⭐️ Оцінювання:
- **1 бал** — за кожен знайдений підступ
- **+2 бали** — за кожен виправлений підступ (будь-яким зручним для вас способом, крім повного `drop()`, окрім одного випадку). Не забувайте за пояснення.


In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix


np.random.seed(42)
n = 180

def rand_yes_no(size):
    choices = ['yes','no','YES','No','True','False','1','0',' y ',' n ','t','f','','none']
    return np.random.choice(choices, size=size)

df = pd.DataFrame({
    'battery_power': np.random.randint(1500, 6000, size=n),
    'bluetooth': rand_yes_no(n),
    'weight': np.random.normal(170, 20, size=n).round(1),
    'memory': np.random.choice([32,64,128,256,512], size=n, p=[0.2,0.3,0.3,0.15,0.05]),
    'n_cores': np.random.choice([0,1,2,4,6,8,12,16,20,-1], size=n,
                                p=[0.04,0.16,0.16,0.24,0.12,0.12,0.06,0.04,0.04,0.02]),
    'ram': np.random.choice([2,3,4,6,8,12,16], size=n, p=[0.05,0.05,0.25,0.25,0.25,0.1,0.05]),
    'dual_sim': rand_yes_no(n),
    'wifi': rand_yes_no(n),
    'pixel_height': np.random.randint(720, 4000, size=n),
    'pixel_width': np.random.randint(720, 4000, size=n),
    'price_range': np.random.choice([0,1,2,3,np.nan], size=n, p=[0.22,0.23,0.23,0.22,0.10])
})


for col in ['battery_power','weight','memory','ram','pixel_height','pixel_width']:
    idx = np.random.choice(df.index, size=5, replace=False)
    df.loc[idx, col] = np.nan

print("🧾 Розмір датафрейму:", df.shape)

binary_cols = ['bluetooth', 'dual_sim', 'wifi']
existing = [c for c in binary_cols if c in df.columns]

mapping = {
    'yes': 1, 'no': 0,
    'true': 1, 'false': 0,
    'y': 1, 'n': 0,
    't': 1, 'f': 0,
    '1': 1, '0': 0
}

report_bin = {}
for col in existing:
    s = df[col].astype(str).str.strip().str.lower()
    s = s.replace({'': np.nan, 'none': np.nan, 'nan': np.nan, 'na': np.nan})
    before = s.value_counts(dropna=False).to_dict()

    mapped = s.map(mapping)
    numeric = pd.to_numeric(s, errors='coerce')
    mapped = mapped.where(~(mapped.isna() & numeric.notna()), numeric)

    mapped = mapped.fillna(0).round().astype(int).clip(0,1)
    df[col] = mapped

    after = df[col].value_counts(dropna=False).to_dict()
    report_bin[col] = {'before': before, 'after': after}

print("\n✅ Бінарні фічі оброблено (YES/NO → 1/0).")
for col, r in report_bin.items():
    print(f"[{col}] Before -> After:", r['before'], " -> ", r['after'])

for col in existing:
    bad = set(df[col].unique()) - {0,1}
    assert not bad, f"Колонка {col} містить не-бінарні значення: {bad}"


if 'n_cores' in df.columns:
    df['n_cores'] = pd.to_numeric(df['n_cores'], errors='coerce')
    valid_mask = df['n_cores'].between(1, 16)
    median_cores = df.loc[valid_mask, 'n_cores'].median()
    before_vals = df['n_cores'].value_counts(dropna=False).to_dict()

    df.loc[~valid_mask, 'n_cores'] = median_cores
    df['n_cores'] = df['n_cores'].fillna(median_cores).round().astype(int)

    after_vals = df['n_cores'].value_counts(dropna=False).to_dict()
    print("\n🛠️ n_cores: фіксація діапазону [1..16].")
    print("Before -> After:", before_vals, " -> ", after_vals)



num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
obj_cols = df.select_dtypes(include=['object']).columns.tolist()

for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

for col in obj_cols:
    if df[col].isna().any():
        mode_val = df[col].mode(dropna=True)
        df[col] = df[col].fillna(mode_val.iloc[0] if not mode_val.empty else 'unknown')

df['total_pixels'] = df['pixel_height'] * df['pixel_width']
df['ppi_proxy'] = (df['total_pixels'] ** 0.5)  # грубий сурогат щільності
df['battery_per_weight'] = df['battery_power'] / (df['weight'].replace(0, np.nan))
df['mem_ram'] = df['memory'] * df['ram']


for col in ['total_pixels','ppi_proxy','battery_per_weight','mem_ram']:
    df[col] = df[col].fillna(df[col].median())


train_df = df[~df['price_range'].isna()].copy()
train_df['price_range'] = train_df['price_range'].astype(int)

X = train_df.select_dtypes(include=[np.number]).drop(columns=['price_range'], errors='ignore')
y = train_df['price_range'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred_cont = reg.predict(X_test)
y_min, y_max = int(y.min()), int(y.max())
y_pred_cls = np.clip(np.rint(y_pred_cont), y_min, y_max).astype(int)

print("\n— Метрики: ЛІНІЙНА регресія як класифікація —")
print("Accuracy:", round(accuracy_score(y_test, y_pred_cls), 4))
print("F1 (weighted):", round(f1_score(y_test, y_pred_cls, average='weighted'), 4))
print(confusion_matrix(y_test, y_pred_cls))
print(classification_report(y_test, y_pred_cls, zero_division=0))


pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(max_iter=3000, class_weight='balanced', solver='lbfgs'))
])

param_grid = {'clf__C': [0.1, 0.5, 1.0, 2.0, 5.0]}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
gs = GridSearchCV(pipe, param_grid, scoring='f1_weighted', cv=cv, n_jobs=-1)
gs.fit(X, y)

best = gs.best_estimator_
print("\nBest params (LogReg):", gs.best_params_)

best.fit(X_train, y_train)
y_pred_log = best.predict(X_test)

print("\n— Метрики: ЛОГІСТИЧНА регресія (scaled + balanced) —")
print("Accuracy:", round(accuracy_score(y_test, y_pred_log), 4))
print("F1 (weighted):", round(f1_score(y_test, y_pred_log, average='weighted'), 4))
print(confusion_matrix(y_test, y_pred_log))
print(classification_report(y_test, y_pred_log, zero_division=0))

rf = RandomForestClassifier(
    n_estimators=400, max_depth=None, class_weight='balanced_subsample',
    random_state=42, n_jobs=-1
)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("\n— Метрики: RandomForest —")
print("Accuracy:", round(accuracy_score(y_test, y_pred_rf), 4))
print("F1 (weighted):", round(f1_score(y_test, y_pred_rf, average='weighted'), 4))
print(confusion_matrix(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf, zero_division=0))


if df['price_range'].isna().any():
    future_X = df.loc[df['price_range'].isna(), X.columns]
    future_pred = best.predict(future_X)
    print("\n📦 Передбачення для рядків без price_range (логістична регресія):")
    print(pd.Series(future_pred).value_counts().sort_index())


🧾 Розмір датафрейму: (180, 11)

✅ Бінарні фічі оброблено (YES/NO → 1/0).
[bluetooth] Before -> After: {'yes': 37, 'no': 26, nan: 25, '0': 16, 'y': 14, '1': 12, 'f': 12, 'false': 12, 'n': 9, 'true': 9, 't': 8}  ->  {0: 100, 1: 80}
[dual_sim] Before -> After: {nan: 34, 'yes': 25, 'no': 21, '0': 19, 'y': 18, 't': 16, 'true': 14, 'n': 10, 'f': 10, 'false': 8, '1': 5}  ->  {0: 102, 1: 78}
[wifi] Before -> After: {nan: 32, 'no': 30, 'yes': 24, 'f': 17, 't': 16, 'y': 11, 'n': 11, '1': 11, 'false': 11, 'true': 9, '0': 8}  ->  {0: 109, 1: 71}

🛠️ n_cores: фіксація діапазону [1..16].
Before -> After: {4: 51, 1: 33, 2: 25, 8: 23, 6: 14, 16: 12, 12: 10, 0: 5, 20: 4, -1: 3}  ->  {4: 63, 1: 33, 2: 25, 8: 23, 6: 14, 16: 12, 12: 10}

— Метрики: ЛІНІЙНА регресія як класифікація —
Accuracy: 0.3611
F1 (weighted): 0.2701
[[0 4 4 0]
 [0 7 5 0]
 [0 3 6 0]
 [0 2 5 0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.44      0.58

*_Знайдені* підступи:_
1. Бінарні фічі → значення "yes", "No", "True", "f", nan
Виправлено: мапінгом у 0 / 1.

2. n_cores → аномальні 0, -1, 20.
Виправлено: замінено на медіану валідних значень.

3.Пропуски у числових → наприклад, battery_power та ram.
Виправлено: заповнено медіаною.

4.price_range → пропуски для майбутніх телефонів.
Виправлено: залишено як є, ці рядки не потрапляють у тренування.

5. Дивні співвідношення роздільної здатності екрану → надто низькі або високі PPI.
Виправлено: створено допоміжну фічу ppi_proxy для контролю.

## Завдання 2: Лінійна регресія — 5 балів
- Побудуйте лінійну регресію для прогнозу `price_range`
- Поясніть які фічі ви використали і чому (тут вам допоможе пророблена робота з завдання 1)
- Уважно прогляньте, що ваша модель передбачає ваше модель, який там дип даних?
- Використайте хоча б одну метрику для оцінки перформансу вашої моделі. [Cheat Sheet model evaluation](https://medium.com/ml-cheat-sheet/machine-learning-evaluation-metrics-b89b8832e275)


In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, f1_score, classification_report


np.random.seed(42)
n = 220

def rand_yes_no(size):

    choices = ['yes','no','YES','No','True','False','1','0',' y ',' n ','t','f','','none']
    return np.random.choice(choices, size=size)

df = pd.DataFrame({
    'battery_power': np.random.randint(1500, 6000, size=n),
    'bluetooth': rand_yes_no(n),
    'weight': np.random.normal(170, 20, size=n).round(1),
    'memory': np.random.choice([32,64,128,256,512], size=n, p=[0.2,0.3,0.3,0.15,0.05]),
    'n_cores': np.random.choice([0,1,2,4,6,8,12,16,20,-1], size=n,
                                p=[0.03,0.17,0.17,0.24,0.12,0.12,0.07,0.04,0.03,0.01]),
    'ram': np.random.choice([2,3,4,6,8,12,16], size=n, p=[0.05,0.05,0.25,0.25,0.25,0.1,0.05]),
    'dual_sim': rand_yes_no(n),
    'wifi': rand_yes_no(n),
    'pixel_height': np.random.randint(720, 4000, size=n),
    'pixel_width': np.random.randint(720, 4000, size=n),

    'price_range': np.random.choice([0,1,2,3,np.nan], size=n, p=[0.25,0.25,0.25,0.15,0.10])
})


for col in ['battery_power','weight','memory','ram','pixel_height','pixel_width']:
    idx = np.random.choice(df.index, size=6, replace=False)
    df.loc[idx, col] = np.nan


binary_cols = ['bluetooth', 'dual_sim', 'wifi']
mapping = {'yes':1,'no':0,'true':1,'false':0,'y':1,'n':0,'t':1,'f':0,'1':1,'0':0}
for col in [c for c in binary_cols if c in df.columns]:
    s = df[col].astype(str).str.strip().str.lower()
    s = s.replace({'': np.nan, 'none': np.nan, 'nan': np.nan, 'na': np.nan})
    mapped = s.map(mapping)
    numeric = pd.to_numeric(s, errors='coerce')
    mapped = mapped.where(~(mapped.isna() & numeric.notna()), numeric)
    df[col] = mapped.fillna(0).round().astype(int).clip(0, 1)


if 'n_cores' in df.columns:
    df['n_cores'] = pd.to_numeric(df['n_cores'], errors='coerce')
    valid = df['n_cores'].between(1, 16)
    med = df.loc[valid, 'n_cores'].median()
    df.loc[~valid, 'n_cores'] = med
    df['n_cores'] = df['n_cores'].fillna(med).round().astype(int)


num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())


df_train = df[df['price_range'].notna()].copy()
df_train['price_range'] = df_train['price_range'].astype(int)


df_train['total_pixels']      = df_train['pixel_height'] * df_train['pixel_width']
df_train['ppi_proxy']         = (df_train['total_pixels'] ** 0.5)
df_train['battery_per_weight']= df_train['battery_power'] / df_train['weight'].replace(0, np.nan)
df_train['mem_ram']           = df_train['memory'] * df_train['ram']
for c in ['total_pixels','ppi_proxy','battery_per_weight','mem_ram']:
    df_train[c] = df_train[c].fillna(df_train[c].median())


features = [
    'battery_power','ram','memory','n_cores','bluetooth','dual_sim','wifi',
    'ppi_proxy','battery_per_weight','mem_ram'
]
X = df_train[features]
y = df_train['price_range']


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


reg = LinearRegression()
reg.fit(X_train, y_train)


y_pred_cont = reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred_cont)
r2  = r2_score(y_test, y_pred_cont)
print("=== Лінійна регресія (регресійні метрики) ===")
print(f"MSE: {mse:.4f}")
print(f"R² : {r2:.4f}")
print("y_pred (min..max):", float(y_pred_cont.min()), "→", float(y_pred_cont.max()))

y_min, y_max = int(y.min()), int(y.max())
y_pred_cls = np.clip(np.rint(y_pred_cont), y_min, y_max).astype(int)

acc = accuracy_score(y_test, y_pred_cls)
f1w = f1_score(y_test, y_pred_cls, average='weighted')
print("\n=== Лінійна регресія як класифікація (rounded+clipped) ===")
print(f"Accuracy: {acc:.4f}")
print(f"F1 (weighted): {f1w:.4f}")
print("\nClassification report:\n", classification_report(y_test, y_pred_cls, zero_division=0))


coef_df = pd.DataFrame({'feature': features, 'coef': reg.coef_}).sort_values('coef', ascending=False)
print("\nКоефіцієнти моделі (вищі -> сильніший вплив на збільшення ціни):")
print(coef_df)

=== Лінійна регресія (регресійні метрики) ===
MSE: 1.0612
R² : 0.0099
y_pred (min..max): 0.7030978104060119 → 1.5886956767562972

=== Лінійна регресія як класифікація (rounded+clipped) ===
Accuracy: 0.2500
F1 (weighted): 0.1518

Classification report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        12
           1       0.26      0.71      0.38        14
           2       0.17      0.09      0.12        11
           3       0.00      0.00      0.00         7

    accuracy                           0.25        44
   macro avg       0.11      0.20      0.13        44
weighted avg       0.13      0.25      0.15        44


Коефіцієнти моделі (вищі -> сильніший вплив на збільшення ціни):
              feature      coef
4           bluetooth  0.190066
6                wifi  0.026134
8  battery_per_weight  0.001359
9             mem_ram  0.000020
7           ppi_proxy -0.000012
0       battery_power -0.000121
2              memory 

### Завдання 3. Класифікація — 10 балів

- Побудуйте класифікатор (довільний,але рекомендовано `LogisticRegression`)
- Навчайте модель на тих рядках, де `price_range` присутнє
- Застосуйте її на тих, де значення **відсутнє**
- Додайте колонку `price_range_predicted` і збережіть в неї отримані передбачення.

🧠 Обов'язково поясніть:
- Які фічі використовували?
- Наскільки точний результат? Використайте метрики оцінки.

In [39]:

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix


def ensure_df():
    global df
    if 'df' in globals():
        return
    np.random.seed(42)
    n = 220
    def rand_yes_no(size):
        choices = ['yes','no','YES','No','True','False','1','0',' y ',' n ','t','f','','none']
        return np.random.choice(choices, size=size)
    _df = pd.DataFrame({
        'battery_power': np.random.randint(1500, 6000, size=n),
        'bluetooth': rand_yes_no(n),
        'weight': np.random.normal(170, 20, size=n).round(1),
        'memory': np.random.choice([32,64,128,256,512], size=n, p=[0.2,0.3,0.3,0.15,0.05]),
        'n_cores': np.random.choice([0,1,2,4,6,8,12,16,20,-1], size=n,
                                    p=[0.03,0.17,0.17,0.24,0.12,0.12,0.07,0.04,0.03,0.01]),
        'ram': np.random.choice([2,3,4,6,8,12,16], size=n, p=[0.05,0.05,0.25,0.25,0.25,0.1,0.05]),
        'dual_sim': rand_yes_no(n),
        'wifi': rand_yes_no(n),
        'pixel_height': np.random.randint(720, 4000, size=n),
        'pixel_width': np.random.randint(720, 4000, size=n),
        'price_range': np.random.choice([0,1,2,3,np.nan], size=n, p=[0.25,0.25,0.25,0.15,0.10])
    })

    for c in ['battery_power','weight','memory','ram','pixel_height','pixel_width']:
        idx = np.random.choice(_df.index, size=6, replace=False)
        _df.loc[idx, c] = np.nan
    df = _df

ensure_df()


binary_cols = ['bluetooth', 'dual_sim', 'wifi']
mapping = {'yes':1,'no':0,'true':1,'false':0,'y':1,'n':0,'t':1,'f':0,'1':1,'0':0}

for col in [c for c in binary_cols if c in df.columns]:
    s = df[col].astype(str).str.strip().str.lower()
    s = s.replace({'': np.nan, 'none': np.nan, 'nan': np.nan, 'na': np.nan})
    mapped = s.map(mapping)
    numeric = pd.to_numeric(s, errors='coerce')
    mapped = mapped.where(~(mapped.isna() & numeric.notna()), numeric)
    df[col] = mapped.fillna(0).round().astype(int).clip(0, 1)

if 'n_cores' in df.columns:
    df['n_cores'] = pd.to_numeric(df['n_cores'], errors='coerce')
    valid = df['n_cores'].between(1, 16)
    med = df.loc[valid, 'n_cores'].median()
    df.loc[~valid, 'n_cores'] = med
    df['n_cores'] = df['n_cores'].fillna(med).round().astype(int)


for c in df.select_dtypes(include=[np.number]).columns:
    df[c] = df[c].fillna(df[c].median())


df['total_pixels']       = df['pixel_height'] * df['pixel_width']
df['ppi_proxy']          = (df['total_pixels'] ** 0.5)
df['battery_per_weight'] = df['battery_power'] / df['weight'].replace(0, np.nan)
df['mem_ram']            = df['memory'] * df['ram']
for c in ['total_pixels','ppi_proxy','battery_per_weight','mem_ram']:
    df[c] = df[c].fillna(df[c].median())


known = df[df['price_range'].notna()].copy()
known['price_range'] = known['price_range'].astype(int)

features = [
    'battery_power','ram','memory','n_cores',
    'bluetooth','dual_sim','wifi',
    'ppi_proxy','battery_per_weight','mem_ram'
]
X = known[features]
y = known['price_range'].astype(int)

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(max_iter=3000, class_weight='balanced', solver='lbfgs'))
])


param_grid = {'clf__C': [0.1, 0.5, 1.0, 2.0]}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
gs = GridSearchCV(pipe, param_grid, scoring='f1_weighted', cv=cv, n_jobs=-1)
gs.fit(X_tr, y_tr)

best = gs.best_estimator_
y_pred = best.predict(X_te)

print("Best params:", gs.best_params_)
print("\n— Оцінка на hold-out —")
print("Accuracy:", round(accuracy_score(y_te, y_pred), 4))
print("F1 (weighted):", round(f1_score(y_te, y_pred, average='weighted'), 4))
print(confusion_matrix(y_te, y_pred))
print(classification_report(y_te, y_pred, zero_division=0))


best.fit(X, y)

unknown_mask = df['price_range'].isna()
if unknown_mask.any():
    X_future = df.loc[unknown_mask, features]
    df.loc[unknown_mask, 'price_range_predicted'] = best.predict(X_future)


df.loc[~unknown_mask, 'price_range_predicted'] = best.predict(X)

print("\n📦 Підсумок предиктів для рядків без price_range:")
if unknown_mask.any():
    print(df.loc[unknown_mask, 'price_range_predicted'].value_counts().sort_index())
else:
    print("Немає рядків із відсутнім price_range.")


print("\nПерших 10 рядків з колонкою 'price_range_predicted':")
print(df[['price_range', 'price_range_predicted']].head(10))

Best params: {'clf__C': 1.0}

— Оцінка на hold-out —
Accuracy: 0.3864
F1 (weighted): 0.3793
[[7 3 2 0]
 [6 3 1 4]
 [1 1 5 4]
 [1 1 3 2]]
              precision    recall  f1-score   support

           0       0.47      0.58      0.52        12
           1       0.38      0.21      0.27        14
           2       0.45      0.45      0.45        11
           3       0.20      0.29      0.24         7

    accuracy                           0.39        44
   macro avg       0.37      0.38      0.37        44
weighted avg       0.39      0.39      0.38        44


📦 Підсумок предиктів для рядків без price_range:
Немає рядків із відсутнім price_range.

Перших 10 рядків з колонкою 'price_range_predicted':
   price_range  price_range_predicted
0          3.0                    3.0
1          1.0                    1.0
2          3.0                    0.0
3          0.0                    2.0
4          1.0                    1.0
5          1.0                    3.0
6          0.0     

### Завдання 4. Кластеризація — 10 балів

- Проведіть кластеризацію (довільну яка вам подобається, але рекомендовано — `KMeans`)
- Навчайте модель на тих даних, де є `price_range`
- Визначте і поділить дані на оптимальну кількість кластерів (з поясненням)
- Потім за допомогою `.predict()` класифікуйте ті рядочки, де `price_range` відсутній
- Додайте колонку: `cluster_suggested` і запишіть туди результат передбачення

Тут можна без інтерпретації чи логіки міркувань. Потрібно **правильно обрати дані** для кластеризації.

In [42]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import warnings


def ensure_df():
    global df
    if 'df' in globals():
        return
    np.random.seed(42)
    n = 220
    df_local = pd.DataFrame({
        'battery_power': np.random.randint(1500, 6000, size=n),
        'weight': np.random.normal(170, 20, size=n).round(1),
        'memory': np.random.choice([32,64,128,256,512], size=n, p=[0.2,0.3,0.3,0.15,0.05]),
        'n_cores': np.random.choice([0,1,2,4,6,8,12,16,20,-1], size=n,
                                    p=[0.03,0.17,0.17,0.24,0.12,0.12,0.07,0.04,0.03,0.01]),
        'ram': np.random.choice([2,3,4,6,8,12,16], size=n, p=[0.05,0.05,0.25,0.25,0.25,0.1,0.05]),
        'price_range': np.random.choice([0,1,2,3,np.nan], size=n, p=[0.25,0.25,0.25,0.15,0.10])
    })

    for c in ['battery_power','weight','memory','ram']:
        idx = np.random.choice(df_local.index, size=6, replace=False)
        df_local.loc[idx, c] = np.nan
    df = df_local

ensure_df()


features = ['battery_power', 'ram', 'memory', 'n_cores', 'weight']
missing = [c for c in features if c not in df.columns]
if missing:
    raise KeyError(f"Відсутні колонки у df: {missing}")


df['n_cores'] = pd.to_numeric(df['n_cores'], errors='coerce')
valid = df['n_cores'].between(1, 16)
med_cores = df.loc[valid, 'n_cores'].median()
df.loc[~valid, 'n_cores'] = med_cores
df['n_cores'] = df['n_cores'].fillna(med_cores).round().astype(int)

for c in ['battery_power','ram','memory','weight']:
    df[c] = pd.to_numeric(df[c], errors='coerce').fillna(df[c].median())


df_with_price = df[df['price_range'].notna()].copy()
df_without_price = df[df['price_range'].isna()].copy()


if len(df_with_price) < 11:
    warnings.warn("Замало рядків із відомим price_range для стабільного підбору K. "
                  "Зменшіть верхню межу діапазону K або використайте інші дані.")


scaler = StandardScaler()
X_train = scaler.fit_transform(df_with_price[features])
X_predict = scaler.transform(df_without_price[features]) if len(df_without_price) else None


max_k = min(10, len(df_with_price) - 1)
range_n_clusters = range(2, max(3, max_k + 1))

silhouette_scores = []
inertias = []
for k in range_n_clusters:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X_train)

    score = silhouette_score(X_train, labels)
    silhouette_scores.append((k, score))
    inertias.append((k, km.inertia_))


best_k, best_score = max(silhouette_scores, key=lambda t: (t[1], -t[0]))


kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=10)
kmeans.fit(X_train)


X_all = scaler.transform(df[features])
df['cluster_suggested'] = kmeans.predict(X_all)

print(f"✅ Обрано оптимальне K за silhouette: K={best_k}, score={best_score:.4f}")
print("Перші 10 рядків:\n", df[['price_range', 'cluster_suggested']].head(10))


✅ Обрано оптимальне K за silhouette: K=3, score=0.2211
Перші 10 рядків:
    price_range  cluster_suggested
0          3.0                  0
1          1.0                  0
2          3.0                  0
3          0.0                  2
4          1.0                  0
5          1.0                  0
6          0.0                  0
7          0.0                  0
8          2.0                  1
9          2.0                  0
